# Core Objective

- `recommend`( user ) --> list of movies

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

%run 'movies.ipynb'
%run 'tags.ipynb'
%run 'ratings.ipynb'
%run 'utils.ipynb'

pd.__version__

'0.20.1'

In [48]:
Tags = getTags()
allMovies = getMovies()
Ratings = getRatings()

verbose ==> False


In [49]:
Ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


- Some basic Information retrieval on our data sets.

In [78]:
print('We have %d movies in our current movie set.' %(allMovies['movieId'].count()))

num_of_users = len(Ratings['userId'].unique());
num_of_movies = len(Ratings['movieId'].unique());
num_of_ratings = Ratings.shape[0]

print('We have %d ratings given by %d users for %d movies' % (num_of_ratings, num_of_users, num_of_movies))

filteredRating = Ratings[ Ratings['movieId'].isin(allMovies.movieId)]
print('Only %d ratings belong %d movies'%( filteredRating.shape[0], allMovies['movieId'].count()))

num_of_unique_users = len(filteredRating['userId'].unique())
print('Only %d unique user contributed in %d ratings.' % ( num_of_unique_users, filteredRating.shape[0]))

num_of_users_to_pick = 10;   # NUMBER OF USERS YOU WANT TO PICK FOR ANALYSIS.
users_picked = np.take(filteredRating['userId'].unique(), np.arange(1, num_of_users_to_pick + 1, 1))
rating_belong_to_picked_users = filteredRating[ filteredRating['userId'].isin(users_picked) ]
print('Lets pick %d user to play with.' % ( num_of_users_to_pick))
print('These %d users provided %d ratings'% (num_of_users_to_pick, rating_belong_to_picked_users.shape[0]))


We have 549 movies in our current movie set.
We have 100004 ratings given by 671 users for 9066 movies
Only 19175 ratings belong 549 movies
Only 663 unique user contributed in 19175 ratings.
Lets pick 10 user to play with.
These 10 users provided 224 ratings


In [81]:
_Ratings = rating_belong_to_picked_users.copy(deep=True)
_Ratings.shape

for userId, group in _Ratings.groupby(by=['userId']):
    print('(userId, total_rating) -> (%d, %d)' % (userId, group.shape[0]))

(userId, total_rating) -> (2, 74)
(userId, total_rating) -> (3, 14)
(userId, total_rating) -> (4, 33)
(userId, total_rating) -> (5, 18)
(userId, total_rating) -> (6, 5)
(userId, total_rating) -> (7, 40)
(userId, total_rating) -> (8, 17)
(userId, total_rating) -> (9, 12)
(userId, total_rating) -> (10, 6)
(userId, total_rating) -> (11, 5)


### Prepare a `(user X movie)` matrix.

In [110]:
list_of_movies = allMovies['title'];

print('Utility matrix should have shape (%d, %d)' % (num_of_users_to_pick, len(list_of_movies)))
utilityMatrix = pd.DataFrame( columns= list_of_movies)
utilityMatrix.head()

for index in np.arange(num_of_users_to_pick):
    userId = users_picked[index];
    # Fillup available ratings.
    rating_by_userId = _Ratings[_Ratings['userId'] == userId]['rating']
    utilityMatrix.loc[index] = [ rating_by_userId.mean() for n in range(len(list_of_movies))]
    
utilityMatrix.head(10)

Utility matrix should have shape (10, 549)


title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",Dracula: Dead and Loving It (1995),Balto (1995),Nixon (1995),Cutthroat Island (1995),Casino (1995),Sense and Sensibility (1995),Four Rooms (1995),Ace Ventura: When Nature Calls (1995),Money Train (1995),Get Shorty (1995),Copycat (1995),Assassins (1995),Powder (1995),Leaving Las Vegas (1995),Othello (1995),Now and Then (1995),Persuasion (1995),"City of Lost Children, The Cité des enfants perdus, La (1995)",Shanghai Triad Yao a yao yao dao waipo qiao (1995),Dangerous Minds (1995),Twelve Monkeys <a.k.a. 12 Monkeys> (1995),Babe (1995),Carrington (1995),Dead Man Walking (1995),Across the Sea of Time (1995),It Takes Two (1995),Clueless (1995),"Cry, the Beloved Country (1995)",Richard III (1995),Dead Presidents (1995),Restoration (1995),Mortal Kombat (1995),To Die For (1995),How to Make an American Quilt (1995),Seven <a.k.a. Se7en> (1995),Pocahontas (1995),When Night Is Falling (1995),"Usual Suspects, The (1995)",Mighty Aphrodite (1995),Lamerica (1994),"Big Green, The (1995)",Georgia (1995),Home for the Holidays (1995),"Postman, The <Postino, Il> (1994)","Confessional, The <Confessionnal, Le> (1995)","Indian in the Cupboard, The (1995)",Eye for an Eye (1996),Mr. Holland's Opus (1995),Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996),Two if by Sea (1996),Bio-Dome (1996),Lawnmower Man 2: Beyond Cyberspace (1996),French Twist <Gazon maudit> (1995),Friday (1995),From Dusk Till Dawn (1996),Fair Game (1995),Kicking and Screaming (1995),"Misérables, Les (1995)",Bed of Roses (1996),Screamers (1995),Nico Icon (1995),"Crossing Guard, The (1995)","Juror, The (1996)","White Balloon, The <Badkonake sefid> (1995)",Things to Do in Denver When You're Dead (1995),Antonia's Line <Antonia> (1995),Once Upon a Time... When We Were Colored (1995),Last Summer in the Hamptons (1995),Angels and Insects (1995),White Squall (1996),Dunston Checks In (1996),Black Sheep (1996),Nick of Time (1995),Mary Reilly (1996),Vampire in Brooklyn (1995),Beautiful Girls (1996),Broken Arrow (1996),In the Bleak Midwinter (1995),"Hate <Haine, La> (1995)",Shopping (1994),Heidi Fleiss: Hollywood Madam (1995),City Hall (1996),Bottle Rocket (1996),Mr. Wrong (1996),Unforgettable (1996),Happy Gilmore (1996),"Bridges of Madison County, The (1995)",Muppet Treasure Island (1996),Catwalk (1996),Braveheart (1995),Taxi Driver (1976),Rumble in the Bronx <Hont faan kui> (1995),Before and After (1996),Margaret's Museum (1995),Anne Frank Remembered (1995),"Young Poisoner's Handbook, The (1995)",If Lucy Fell (1996),"Steal Big, Steal Little (1995)","Boys of St. Vincent, The (1992)",Boomerang (1992),Chungking Express <Chung Hing sam lam> (1994),"Star Maker, The <Uomo delle stelle, L'> (1995)",Flirting With Disaster (1996),"NeverEnding Story III, The (1994)",Pie in the Sky (1996),Angela (1995),Frankie Starlight (1995),Jade (1995),Down Periscope (1996),Man of the Year (1995),Up Close and Personal (1996),"Birdcage, The (1996)","Brothers McMullen, The (1995)",Bad Boys (1995),"Amazing Panda Adventure, The (1995)","Basketball Diaries, The (1995)","Awfully Big Adventure, An (1995)",Amateur (1994),Apollo 13 (1995),Rob Roy (1995),"Addiction, The (1995)",Batman Forever (1995),Beauty of the Day <Belle de jour> (1967),Beyond Rangoon (1995),Blue in the Face (1995),Canadian Bacon (1995),Casper (1995),Clockers (1995),Congo (1995),Crimson Tide (1995),Crumb (1994),Desperado (1995),Devil in a Blue Dress (1995),Die Hard: With a Vengeance (1995),"Doom Generation, The (1995)",Feast of July (1995),First Knight (1995),Free Willy 2: The Adventure Home (1995),Hackers (1995),Jeffrey (1995),Johnny Mnemonic (1995),Judge Dredd (1995),Jury Duty (1995),Kids (1995),Living in Oblivion (1995),Lord of Illusions (1995),Love & Human Remains (1993),Mad Love (1995),Mallra